In [ ]:
from __future__ import absolute_import, division
from BashColors import C
from TarfileFunctions import *

import os
from os.path import exists, join
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

contentPath = os.getcwd()
checkpointPath= join(contentPath, 'Checkpoints')
genPath = join(contentPath, 'DataGenerator')
if not exists(genPath):
    tff.extractTarfiles('DataGenerator.tar.gz')
testPath = join(contentPath, 'images')
if not exists(testPath):
    tff.extractTarfiles('images.tar.gz')

import numpy as np
np.set_printoptions(precision=4)

import tensorflow as tf
print(f'tensorflow: {C.BIBlue}{tf.__version__}{C.ColorOff}')

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=2, verbose=1,
    mode='auto', baseline=None, restore_best_weights=True,
    # print('\n',
)

checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpointPath,
    monitor='loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)
print('ok')

In [ ]:
IMAGE_SIZE=(224, 224)
BATCH_SIZE = 4

import tensorflow
from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    genPath,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

validation_ds = image_dataset_from_directory(
    genPath,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    shuffle=True,
    batch_size=BATCH_SIZE)
print()

test_ds = image_dataset_from_directory(
    testPath,
    color_mode='rgb',
    image_size=(224, 224),
    shuffle=False,
    batch_size = BATCH_SIZE)
# clear_output()

TRAIN_STEPS = len(train_ds) // BATCH_SIZE
print('\nTRAIN_STEPS:', TRAIN_STEPS)
VAL_STEPS = len(validation_ds) // BATCH_SIZE
print('VAL_STEPS:', VAL_STEPS)

In [ ]:
INPUT_SHAPE = (224, 224, 3)
DROPOUT = 0.001  # Default to 0.001
ALPHA = 2.0  # Default to 1.0

model  =  tf.keras.applications.mobilenet.MobileNet(   input_shape = INPUT_SHAPE, alpha = ALPHA, depth_multiplier = 16,
    dropout = DROPOUT, include_top = False, weights = None,
    input_tensor = None, pooling = None, classes = 1000,
    classifier_activation = 'softmax'
)

print()

print()
model.summary()

In [ ]:
layerCount=0
for layer in model.layers:
    layerCount+=1
    print(f'{layerCount}. {C.BIBlue}{layer}{C.ColorOff} parameters:{layer.count_params()}')